# Correlation, Regression, and Least Squares

Run the cell below to prepare the notebook.

In [ ]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
import pandas as pd
# Safe settings for Pandas.
pd.set_option('mode.chained_assignment', 'raise')

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# This silences some warnings.
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Stuff for fancy widgets.
from matplotlib import patches
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

# Automated tests.
from client.api.notebook import Notebook
ok = Notebook('on_regression.ok')

## 1. Evaluating basketball game predictions


### A brief introduction to sports betting

In a basketball game, each team scores some number of points.  Conventionally,
the team playing at its own arena is called the "home team," and the other
team is called the "away team."  The winner is the team with the most points.

We can summarize what happened in a game by the "**outcome**", defined as
**the away team's score minus the home team's score**:

$$\text{outcome} = \text{points scored by the away team} - \text{points scored
by the home team}$$

If this number is positive, the away team won.  If it's negative, the home
team won.

Casinos in Las Vegas offer bets on the outcomes of National Basketball
Association (NBA) games.  One kind of bet works like this:

1. The casino decides on a number called the "spread."
2. You can bet 11 dollars that the outcome will be above the spread, or 11
   dollars that the outcome will be below the spread.
3. After the game, you end up with 21 dollars if you guessed correctly, and
   0 if you guessed incorrectly.

The analysts at the casino try to choose the "spread* value so that (according
to their analysis of the teams) there is a 50% chance that the outcome will be
below that value, and a 50% chance that the outcome will be above that value.

**The spread is the casino's best guess at the outcome (the away team's score
minus the home team's score).**

The table `spreads` contains spreads from the betting website
[Covers](http://www.covers.com) from every game in the 2014 NBA season, plus
actual game outcomes.

In [ ]:
spreads = pd.read_csv("spreads.csv")
spreads.head()

Here's a scatter plot of the outcomes and spreads, with the spreads on the
horizontal axis.

In [ ]:
spreads.plot.scatter("Spread", "Outcome")

### Question 1

For reflection; why do you think that neither the spread and outcome are ever
0 (aside from a single case of the spread being 0)?

Let's investigate how well the casinos are predicting game outcomes.

One question we can ask is: Is the casino's prediction correct on average? In
other words, for every value of the spread, is the average outcome of games
assigned that spread equal to the spread? If not, the casino would apparently
be making a systematic error in its predictions.


### Question 2

Among games with a spread around 5, from 3.5 to 6.5 (including both 3.5 and
6.5), what was the average outcome?

*Hint:* Consider doing this by [combining Boolean arrays](https://matthew-brett.github.io/cfd2020/mean-slopes/combining_boolean_arrays).

In [ ]:
spreads_around_5 = ...
spread_5_outcome_average = ...
print("Average outcome for spreads around 5:", spread_5_outcome_average)

In [ ]:
_ = ok.grade('q1_2')

### Question 3

If the average outcome for games with any given spread turned out to be
exactly equal to that spread, what would the slope and intercept of the linear
regression line be, in original units?

In [ ]:
expected_slope_for_equal_spread = ...
expected_intercept_for_equal_spread = ...

In [ ]:
_ = ok.grade('q1_3')

### Question 4

Fix the `standard_units` function below.  It should take an array of numbers
as its argument and return an array of those numbers in standard units.

In [ ]:
def standard_units(nums):
    """ Return array where every value in nums is converted to standard units.
    """
    return nums / np.std(nums) - np.mean(nums)

In [ ]:
_ = ok.grade('q1_4')

### Question 5

Compute the correlation between outcomes and spreads using the `standard_units` function.

In [ ]:
spread_r = ...
spread_r

In [ ]:
_ = ok.grade('q1_5')

### Question 6a

In fact, you can use the correlation you have just calculated to give the
least-squares linear regression line that predicts outcomes from spreads, in
original units.

To do this, you need to:

* Divide the r value by the standard deviation of the original x values (here
  the Spread values), then
* Multiply the result by the standard deviation of the original y values (here
  the Outcome values).

Do that now to calculate the least-squares slope in original units.

In [ ]:
spread_slope = ...
spread_slope

In [ ]:
_ = ok.grade('q1_6')

### Question 6b

Confirm this is very close to the least-squares slope by using `minimize`.
Here are your [usual tools](https://matthew-brett.github.io/cfd2020/mean-slopes/using_minimize):

In [ ]:
from scipy.optimize import minimize

def ss_any_line(c_s, x_values, y_values):
    c, s = c_s
    predicted = c + x_values * s
    error = y_values - predicted
    return np.sum(error ** 2)

Now use `minimize` to re-calculate the least-squares slope:

In [ ]:
minimized_spread_slope = ...
minimized_spread_slope

In [ ]:
_ = ok.grade('q1_6b')

### Question 7

For the "best fit" line that estimates the average outcome from the spread,
the slope is less than 1. Does knowing the slope alone tell you whether the
average spread was higher than the average outcome? If so, set the variable
name below to `True`. If you think you need more information than just the
slope of the regression line to answer that question, then respond `False`.
Briefly justify your answer below.

In [ ]:
slope_implies_average_spread_above_average_outcome = ...

*Write your answer here, replacing this text.*

In [ ]:
_ = ok.grade('q1_7')

## 2. Finding the Least Squares Regression Line

In this exercise, you'll work with a small invented data set.  Run the next
cell to generate the dataset `d` and see a scatter plot.

In [ ]:
# Run this cell.
d = pd.DataFrame()
d['x'] = [0,  1,  2,  3,  4]
d['y'] = [1, .5, -1,  2, -3]
d.plot.scatter('x', 'y')

### Question 1 (Ungraded, but you'll need the result later)

Running the cell below will generate sliders that control the slope and
intercept of a line through the scatter plot.  When you adjust a slider, the
line will move.

By moving the line around, make your best guess at the least-squares
regression line.  (It's okay if your line isn't exactly right, as long as it's
reasonable.)

**Note:** Python will probably take about a second to redraw the plot each
time you adjust the slider.  We suggest clicking the place on the slider you
want to try and waiting for the plot to be drawn; dragging the slider handle
around will cause a long lag.

In [ ]:
def plot_line(intercept, slope):
    plt.figure(figsize=(5,5))

    endpoints = np.array([-2, 7])
    p = plt.plot(endpoints, intercept + slope * endpoints,
                 color='orange',
                 label='Proposed line')

    plt.scatter(d['x'], d['y'], color='blue', label='Points')

    plt.xlim(-4, 8)
    plt.ylim(-6, 6)
    plt.gca().set_aspect('equal', adjustable='box')

    plt.legend(bbox_to_anchor=(1.8, .8))
    plt.show()

interact(plot_line,
         intercept=widgets.FloatSlider(min=-4, max=4, step=.1),
         slope=widgets.FloatSlider(min=-4, max=4, step=.1));

<div class="hide">\pagebreak</div>

You can probably find a reasonable-looking line by just eyeballing it.  But
remember: the least-squares regression line minimizes the mean of the squared
errors made by the line for each point.  Your eye might not be able to judge
squared errors very well.


### A note on mean and total squared error

It is common to think of the least-squares line as the line with the least
*mean* squared error (or the square root of the mean squared error).

But that it doesn't matter whether you minimize the *mean* squared error or
the *total* squared error.  You'll get the same best line in either case.

That's because the total squared error is just the mean squared error
multiplied by the number of points (`len(d)`).  So if one line gets a better
total squared error than another line, then it also gets a better mean squared
error.  In particular, the line with the smallest total squared error is also
better than every other line in terms of mean squared error.  That makes it
the least squares line.

**Minimizing the total squared error minimizes the mean squared error as
well.**


### Question 2 (Ungraded, but you'll need the result later)

The next cell produces a more useful plot.  Use it to find a line that's
closer to the least-squares regression line, keeping the above note in mind.

In [ ]:
def plot_line_and_errors(intercept, slope):
    plt.figure(figsize=(5,5))
    points = np.array([-2, 7])
    p = plt.plot(points, intercept + slope * points,
                 color='orange',
                 label='Proposed line')
    ax = p[0].axes

    predicted_ys = intercept + slope * d['x']
    diffs = predicted_ys - d['y']
    for i in np.arange(len(d)):
        x = d['x'].iloc[i]
        y = d['y'].iloc[i]
        diff = diffs.iloc[i]

        if diff > 0:
            bottom_left_x = x
            bottom_left_y = y
        else:
            bottom_left_x = x + diff
            bottom_left_y = y + diff

        ax.add_patch(patches.Rectangle(
            [bottom_left_x, bottom_left_y],
            abs(diff), abs(diff),
            color='red', alpha=.3,
            label=('Squared error' if i == 0 else None)))
        plt.plot([x, x], [y, y + diff], color='red', alpha=.6,
                 label=('Error' if i == 0 else None))

    plt.scatter(d['x'], d['y'], color='blue', label='Points')

    plt.xlim(-4, 8)
    plt.ylim(-6, 6)
    plt.gca().set_aspect('equal', adjustable='box')

    plt.legend(bbox_to_anchor=(1.8, .8))
    plt.show()

interact(plot_line_and_errors,
         intercept=widgets.FloatSlider(min=-4, max=4, step=.1),
         slope=widgets.FloatSlider(min=-4, max=4, step=.1));

### Question 3

Describe the visual criterion you used to find a line in question 2.  (For
example, a possible (but incorrect) answer is, "I tried to make the red line
for the bottom-right point as small as possible.")

*Write your answer here, replacing this text.*

### Question 4

We can say that a point influences the line by how much the line would move if
the point were removed from the data set. Does the point at (3, 2) have more
or less influence than any other point on the location of the line?

*Write your answer here, replacing this text.*

Now, let's have Python find this line for us.  When we use `minimize`, Python
goes through a process similar to the one you might have used in question 2.


### Question 5

Define a function called `total_squared_error`.  It should take two numbers as arguments:

1. the intercept of some potential line
2. the slope of some potential line

It should return the total squared error when we use that line to make
predictions for the dataset `d`.  You can use `d` inside the function -
remember, function world can see the variables in the notebook, it just can't
change what the variables in the notebook point to.

**Hint** - you can use the `ss_any_line` function above to make your life
easier.

In [ ]:
def total_squared_error(intercept, slope):
    ...

In [ ]:
_ = ok.grade('q2_5')

### Question 6

What is the total squared error for the line you found by "eyeballing" the
errors in Question 1?  What about Question 2, where you made a guess that was
"aided" by a visualization of the squared error?  (It's okay if the error went
up, but for many students, the error will go down when using the visual aid.)

In [ ]:
eyeballed_error = ...
aided_error = ...
print("Eyeballed error:", eyeballed_error)
print("Aided error:", aided_error)

In [ ]:
_ = ok.grade('q2_6')

#### Question 7

Use `minimize` to find the actual slope and intercept of the least-squares regression line.

*Hint*: your task will be easier if you use the general `ss_any_line` function,
rather than your less general `total_squared_error` function above.


In [ ]:
#- The staff solution used 1 line of code above here.
intercept_from_minimize = ...
slope_from_minimize = ...
print("Least-squares regression line: predicted_y =",
      intercept_from_minimize,
      "+",
      slope_from_minimize,
      '* x')

In [ ]:
_ = ok.grade('q2_7')

### Question 8

What was the total squared error for that line?

In [ ]:
best_total_squared_error = ...
best_total_squared_error

In [ ]:
_ = ok.grade('q2_8')

Finally, run the following cell to plot this "best fit" line and its errors:

In [ ]:
plot_line_and_errors(intercept_from_minimize, slope_from_minimize)

## 3. Triple Jump Distances vs. Vertical Jump Heights

Does skill in one sport imply skill in a related sport?  The answer might be
different for different activities.  Let us find out whether it's true for the
[triple jump](https://en.wikipedia.org/wiki/Triple_jump) (an horizontal jump
similar to a long jump) and the vertical jump.  Since we're learning about
linear regression, we will look specifically for a *linear* association
between skill in the two sports.

The following data was collected by observing 40 college level soccer players.
Each athlete's distance in both jump activities was measured in centimeters.
Run the cell below to load the data.

In [ ]:
# Run this cell to load the data
jumps = pd.read_csv('triple_vertical.csv')
jumps.head()

### Question 1

Before running a regression, it's important to see what the data look like,
because our eyes are good at picking out unusual patterns in data.  Draw a
scatter plot with the triple jump distances on the horizontal axis and the
vertical jump heights on vertical axis.

In [ ]:
...

**Question 2** If you were to calculate the *correlation coefficient* `r`, do
you think it would be closer to 0, .5, or -.5? Explain.

*Write your answer here, replacing this text.*

### Question 3

Create a function called `regression_parameters`. It takes as its argument a
table with two columns.  The first column is the x-axis, and the second column
is the y-axis.  It should compute the intercept and slope of the regression
line that predicts the second column from the first, in original units
(centimeters).  It should return an array with two elements: the intercept of
the regression line, and the slope of the regression line.

*Hint*: remember your [Pandas
indexing](https://matthew-brett.github.io/cfd2020/data-frames/df_series_arrays).

*Hint*: use a flat line with intercept 0, slope 0 as your initial guess for the
line.

In [ ]:
def regression_parameters(t):
    ...
    #- Our solution had 1 line above this one
    intercept = ...
    slope = ...
    return np.array([intercept, slope])

# When your function is finished, the next lines should
# compute the regression line predicting vertical jump
# distances from triple jump distances.
parameters = regression_parameters(jumps)
print('Intercept:', parameters[0])
print('Slope:', parameters[1])

In [ ]:
_ = ok.grade('q3_3')

### Question 4

Let's use `regression_parameters` to predict what certain athletes' vertical jump heights would be given their triple jump distances.

The world record for the triple jump distance is 18.29 *meters* by Johnathan
Edwards. What's our prediction for what Edwards' vertical jump would be?

In [ ]:
triple_record_vert_est = ...
print("Predicted vertical jump distance: {:f} centimeters".format(triple_record_vert_est))

In [ ]:
_ = ok.grade('q3_4')

### Question 5

Do you expect this estimate to be accurate within a few centimeters? Why or
why not?

*Write your answer here, replacing this text.*

## 4. Testing Relationship Ratings

Throughout this part of the homework, we will be interested in learning how
different factors can contribute to the rating couples give their
relationship, on a scale of 1-5 (1 being most satisfied, and 5 being the
least).

To do this, we will be looking at the table that is loaded below.

**Note:** The data in this table is quite dated; for example, it only includes
heterosexual relationships. A better dataset would contain a more accurate
sample of couples. When drawing conclusions from data, it is important to be
aware of biases that exist in the dataset.

In [ ]:
couples = pd.read_csv('couples.csv')
couples.head()

Looking at the table above, we note that we should be able to test whether
different factors can affect the relationship rating, from age to education.
To begin, we are first going to investigate whether *marital status* has an
effect on relationship rating. In order to accomplish this, we are going to do
a permutation test!


### Question 1

State the *null hypothesis* for such a test (assume we aren't trying to figure
out ***how*** marital status affects relationship rating, just whether or not
it does).

Remember the *null hypothesis* is a specification of the ideal world in which
there is *not any* effect.


**Answer:**

Null hypothesis:


### Question 2

Before we continue, we will have to talk about the notion of a contingency
table.

Define `contingency` to be a data frame where the *rows* represent unique
values of the relationship ratings, the *columns* represent the unique values
of marital status present in the table, and the values in each cell represent
the number of individuals with a given marital status and relationship rating.

**Hint**: you already know a very useful Pandas function that does this.

In [ ]:
contingency = ...
contingency

In [ ]:
_ = ok.grade('q4_2')

### Question 3

It is difficult to identify a pattern in the above table, since our table has
so many more married people than it does unmarried. To rectify this, we will
instead compute proportions.

Assign `contingency_prop` to a table with the same columns and rows, but the
values are instead in proportions, i.e. the value in the cell corresponding to
a relationship rating of 1 and marital status of "married" should be the
proportion of married people who gave their relationship a 1.  In other words,
replace the values by the proportion of all counts in the matching *column*.

In [ ]:
contingency_prop = ...
contingency_prop

In [ ]:
_ = ok.grade('q4_3')

## Question 4

The `married` column of `contingency_prop` now contains proportions of married
people who gave ratings 1, 2, 3, 4, 5, and the `partner` column has the
equivalent proportions for the not-married people.

We would like to have some measure of the difference `married` and `partner` proportions.

One way of doing this is called *Total Variation Distance* (TVD).

It works like this: for each of the five proportions we subtract the `partner`
proportion from the `married` proportion, and then knock of any minus signs by
taking the absolute value.   We end up with five absolute differences of
proportions.  The TVD is the sum of these five absolute differences divided by
two.

Assign `tvd_marriages` to the Total Variation Distance between the
distributions of married people's relationship ratings and unmarried people's
relationship ratings.

In [ ]:
tvd_marriages = ...
tvd_marriages

In [ ]:
_ = ok.grade('q4_4')

### Question 5

Let's get more general now. Define the function `tvd` which takes in:

- any table like `couples`
- `conditions`: the name of a column like Relationship Rating
- `values`: the name of a column like Marital Status

The function should return the total variation distance between the
distribution of conditions in the first unique value of `values` and the
distribution of conditions in the second unique value of `values` (assume
there are only two unique values)

***Hint***: This looks daunting, but you are just generalizing the code you
have written in the cells above.

In [ ]:
def tvd(t, conditions, values):
    return ...

# This should give the same answer as the cell above.
tvd(couples, 'Relationship Rating', 'Marital Status')

In [ ]:
_ = ok.grade('q4_5')

### Question 6

Now that we have a valid test statistic, fill in the implementation of
`permutation_test` below. It should run a permutation test to check whether
conditions and values are related, where conditions and values are defined as
above.

**Hint** by saying conditions are and values are not related, we are saying
that the pairing between conditions is random.

The inputs to the function are:

- `t`: a table containing our data
- `conditions`: the name of the column in `t` representing our conditions
- `values`: the name of the column in `t` representing our values
- `n_repetitions`: the number of permutations we'd like to test.
- `stat_func`: a test statistic function (such as `tvd`).

The return value should be the p-value of seeing the observed value assuming
the null hypothesis. Fill in the implementation as necessary; we have also
provided code which allows you to see an empirical histogram of the
distribution under the null.

In [ ]:
def permutation_test(t, conditions, values, n_repetitions, stat_func):
    observed_stat = ...
    fake_stats = ...
    # Values as an array, for permuting.
    to_permute = np.array(t[values])

    for i in np.arange(n_repetitions):
        # Permute the values
        ...
        permuted_df = t.copy()
        permuted_df[values] = ...
        fake_stats...

    p_value = ...
    # Show the sampling distribution
    plt.hist(fake_stats, bins=np.arange(-.02, .12, .02))
    plt.title('Empirical distribution of TVD')
    return p_value

permutation_test(couples, 'Relationship Rating', 'Marital Status', 200, tvd)

### Question 7

Based on the above method, should we reject the null hypothesis that the relationship ratings and marital status are uncorrelated, or should we fail to reject? Why?


*Write your answer here, replacing this text.*

## Done

You're finished with the assignment!  Be sure to...

- **run all the tests** (the next cell has a shortcut for that),
- **Save and Checkpoint** from the "File" menu.
- Finally, **restart** the kernel for this notebook, and **run all the
  cells**, to check that the notebook still works without errors.  Use the
  "Kernel" menu, and choose "Restart and run all".  If you find any problems,
  go back and fix them, save the notebook, and restart / run all again, before
  submitting.  When you do this, you make sure that we, your humble markers,
  will be able to mark your notebook.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]